### 0.导入包并设置GPU

In [1]:
import os
import numpy as np

#设置LOG信息显示等级
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
os.environ['CUDA_VISIABLE_DEVICES']='O'

import tensorflow as tf 
print("TensorFlow version:{}".format(tf.__version__))
print("Keras version:{}".format(tf.keras.__version__))


#设置GPU按需申请显存
phy_gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in phy_gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

TensorFlow version:2.3.0
Keras version:2.4.0


### 1.Tensor引入与创建

**Python中常见的数据结构类型有：**
* Python:List
* Numpy:ndarray

这两种数据结构类型可以用来存储大规模数据，但是它们不可以运行在GPU之上。

**在TensorFlow中对应的存放数据的就是tf.Tensor类。**tf.Tensor同Numpy中的ndarray类似，其只是数据容器，里面可以存放多种类型的数据。

TensorFlow中的常见数据类型有：
* int,float,double
* bool
* string


常见创建Tensor的方式有:
* tf.constant()
* tf.Variable()

在不指定数据类型时会根据传入的值自动推导出其数据类型，也可以通过dtype指定数据类型，或者通过tf.cast进行类型转换。

In [2]:
#int32
tf.constant(1)

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [3]:
#string
tf.constant("Hello TensorFlow.")

<tf.Tensor: shape=(), dtype=string, numpy=b'Hello TensorFlow.'>

In [4]:
#float32
tf.constant(1.)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [5]:
#指定dtype为double
tf.constant(1.,dtype=tf.double)

<tf.Tensor: shape=(), dtype=float64, numpy=1.0>

In [6]:
# tf.cast转换数据类型
aa = tf.constant(11)
print(aa.dtype)

bb = tf.cast(aa,tf.float32)
print(bb.dtype)

<dtype: 'int32'>
<dtype: 'float32'>


### 2.EagerExecution

**Tensor的本质其实是一套计算的流程：**

* 在TensorFlow2.0中之所以定义好这个流程之后,计算结果也出来了，是因为EagerExecution这种模式。
* 在EagerExecution之前，Tensor都是只定义(在tf1.x中都是先定义出计算图)，在需要计算的时候再通过session来执行计算。
* 在EagerExecution之后，Tensor定义好后，结果也计算好了。

In [7]:
#检查是否是Eager execution
print("Eager execution is:{}".format(tf.executing_eagerly()))

Eager execution is:True


In [8]:
#a和b都是tf.Tensor，并且在执行完b=a*a后计算结果就已经产生了。
a = tf.constant([[1,2],[3,4]])
b = a*a
a,b,type(a),type(b)

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4]])>,
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[ 1,  4],
        [ 9, 16]])>,
 tensorflow.python.framework.ops.EagerTensor,
 tensorflow.python.framework.ops.EagerTensor)

###  3.Tensor vs ndarray

* Numpy ndarray可以作为整个计算流程的输入，也可以作为整个计算流程的输出。
* 但是注意，**它只能作为输入和输出，进入TensorFlow计算流程之后都要用Tensor。**
* 除非一些之后不在计算流程中使用的Tensor可以通过tf.Tensor.numpy()方法将Tensor以ndarray导出。
* Tensor也有shape属性。
* 可以使用tf.is_tensor()方法测试是否是Tensor。

#### 3.1 ndarray to Tensor

主要有以下几种方法：
* 直接将ndaary传入tf.constant()
* tf.convert_to_tensor()方法

In [9]:
c = np.array([[1,2],[3,4]])
d = tf.constant(c)
c,d,type(c),type(d)

(array([[1, 2],
        [3, 4]]),
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4]])>,
 numpy.ndarray,
 tensorflow.python.framework.ops.EagerTensor)

In [10]:
print(d.shape)
print(tf.is_tensor(c))
print(tf.is_tensor(d))

(2, 2)
False
True


In [11]:
e = np.array([[1,2],[3,4]])
f = tf.convert_to_tensor(c)
e,f,type(e),type(f)

(array([[1, 2],
        [3, 4]]),
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4]])>,
 numpy.ndarray,
 tensorflow.python.framework.ops.EagerTensor)

#### 3.2 Tensor to ndarray

* 直接通过Tensor.numpy()方法来接收就行。

In [12]:
e = d.numpy()
e,type(e)

(array([[1, 2],
        [3, 4]]),
 numpy.ndarray)

### 4.cpu or gpu

In [13]:
print(d.device)
print(tf.config.list_physical_devices('CPU'))
print(tf.config.list_physical_devices('GPU'))

/job:localhost/replica:0/task:0/device:CPU:0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### 5.tf.Variable

* tf.Variable是在Tensor(tf.constant)基础上的进一步封装，因此它本质上也是Tensor。
* tf.constant定义初始化好后是默认无需从外部手动赋值修改的，而这时候如果又想手动修改Tensor的值，于是经过一层对Tensor的封装，可以手动修改的tf.Variable就诞生了。
* Variable可以当成Tensor处理，但是Tensor不一定能当作Variable。

In [14]:
#tf.constant
m = tf.constant([[1,2],[3,4]])
m,type(m)

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4]])>,
 tensorflow.python.framework.ops.EagerTensor)

In [15]:
#tf.Variable
f = tf.Variable([[1,2],[3,4]])
f,type(f)

(<tf.Variable 'Variable:0' shape=(2, 2) dtype=int32, numpy=
 array([[1, 2],
        [3, 4]])>,
 tensorflow.python.ops.resource_variable_ops.ResourceVariable)

In [16]:
#Variable其本质也是Tensor
print(tf.is_tensor(f))
print(f.trainable)

True
True


#### tf.constant不可修改

In [17]:
g = tf.constant(100)
print(g)
g.assign(20)

tf.Tensor(100, shape=(), dtype=int32)


AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

#### tf.Variable可以通过assign方法修改

In [18]:
h = tf.Variable(100)
print(h)
h.assign(50)
print(h)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=100>
<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=50>


### 7.总结

Numpy ndarray VS tf.Tensor VS tf.Variable：

* 从前往后，逐层封装，增加功能，ndarray不可以放在GPU或者TPU上，后两者则可以。
* tf.Tensor定义初始化好后不可手动更改，tf.Variable可以手动更改。
* tf.Variable是面向用户使用，tf.Tensor面向Tensorflow计算流程内部。